In [57]:
# don't forget about day
# if month year exists but couldn't find

import pandas as pd
import psycopg2
import datetime
from psycopg2 import sql
from sqlalchemy import create_engine, text
from dateutil import relativedelta
from loguru import logger

PICKUP_DATETIME_COL = "tpep_pickup_datetime"
TOTAL_AMOUNT_COL = "total_amount"

year = 2020
month = 1
date = datetime.datetime(year, month, 10)
start_date = date.replace(day=1)
end_date = start_date + relativedelta.relativedelta(months=1)
formatted_month_year = date.strftime("%Y-%m")

df = pd.read_parquet(f'yellow_tripdata_{year}-01.parquet', engine='pyarrow')
df = df[
        (df[pickup_datetime_col] >= start_date) & (df[pickup_datetime_col] < end_date)
    ]

df["formatted_pickup_date"] = df[PICKUP_DATETIME_COL].dt.strftime("%Y-%m-%d")
df["hour"] = df[PICKUP_DATETIME_COL].dt.hour
df["month_year"] = formatted_month_year
df["processed_time"] = datetime.datetime.now()


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6404796 entries, 0 to 6405007
Data columns (total 23 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [70]:
zone_df = pd.read_csv('taxi+_zone_lookup.csv')
zone_df

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [93]:
zone_df['Zone'].value_counts()

Governor's Island/Ellis Island/Liberty Island    3
Corona                                           2
Newark Airport                                   1
Ocean Hill                                       1
Parkchester                                      1
                                                ..
Fordham South                                    1
Forest Hills                                     1
Forest Park/Highland Park                        1
Fort Greene                                      1
NV                                               1
Name: Zone, Length: 261, dtype: int64

In [96]:
display(zone_df[zone_df['Zone'] == "Governor's Island/Ellis Island/Liberty Island"])
display(zone_df[zone_df['Zone'] == "Corona"])
# 261 unique()

,LocationID,Borough,Zone,service_zone
102,103,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone
103,104,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone
104,105,Manhattan,Governor's Island/Ellis Island/Liberty Island,Yellow Zone


,LocationID,Borough,Zone,service_zone
55,56,Queens,Corona,Boro Zone
56,57,Queens,Corona,Boro Zone


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6404796 entries, 0 to 6405007
Data columns (total 23 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [92]:
105 in df['PULocationID'].unique()

True

In [61]:
len(df)

6404796

In [100]:
# Aggregation
agg_df = (df
          .groupby(["formatted_pickup_date", "PULocationID","DOLocationID", "hour",])
          .agg(trip_count=(PICKUP_DATETIME_COL, 'count'), avg_amount=(TOTAL_AMOUNT_COL, 'mean'))).reset_index()
# Fill in missing value
# for temp_date in agg_df['formatted_pickup_date'].unique():
#     filtered_df = agg_df[agg_df['formatted_pickup_date'] == temp_date]
#     missing_hours = [hour for hour in range(24) if hour not in filtered_df['hour'].unique()]
#     for hour in missing_hours:
# #         new_row = {'hour': hour, 'pickup_date':temp_date, 'trip_count':0}
#         agg_df = agg_df.append(new_row, ignore_index=True)

agg_df.sort_values(by=['formatted_pickup_date','hour'])

,formatted_pickup_date,PULocationID,DOLocationID,hour,trip_count,avg_amount
32,2020-01-01,3,147,0,1,25.000000
34,2020-01-01,4,4,0,4,5.550000
45,2020-01-01,4,7,0,1,33.360000
49,2020-01-01,4,13,0,1,21.950000
71,2020-01-01,4,37,0,1,26.750000
...,...,...,...,...,...,...
1717626,2020-01-31,264,262,23,2,20.775000
1717657,2020-01-31,264,264,23,52,19.965962
1717660,2020-01-31,264,265,23,1,276.960000
1717675,2020-01-31,265,264,23,1,200.300000


In [101]:
# Assuming formatted_pickup_date is already a datetime object
agg_df['month'] = 

# Group by month, PULocationID, and DOLocationID, and aggregate trip_count into a list
grouped_df = agg_df.groupby(['PULocationID', 'DOLocationID']).agg({'trip_count': list}).reset_index()

# Optionally, you can pivot the DataFrame to have each hour as a separate column
# Assuming your 'hour' column contains integers representing the hour of the day (0-23)
pivot_df = pd.pivot_table(grouped_df, values='trip_count', index=['month', 'PULocationID', 'DOLocationID'], columns='hour', aggfunc='sum', fill_value=[]).reset_index()

# If you prefer the 'hour' column to be part of the MultiIndex
# pivot_df.columns = pd.MultiIndex.from_product([['trip_count'], range(24)], names=['', 'hour'])

# Display the resulting DataFrame
print(pivot_df)

AttributeError: Can only use .dt accessor with datetimelike values

In [49]:
import datetime
date = datetime.date(2022, 9, 1)

month_number = date.month

print(date)
print(f"Month number is: {type(month_number)}")

2022-09-01
Month number is: <class 'int'>
